In [ ]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install pandas
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

plm = "EleutherAI/pythia-160m-deduped"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, Features, Value
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/opendid_valid/opendid_set1.tsv", delimiter='\t',
                       features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import torch
sub_datasets = torch.utils.data.random_split(dataset['train'], [85736,0])
print(len(sub_datasets[0]))
for i in range(4): print(sub_datasets[0][i])

85736
{'fid': '408', 'idx': 642, 'content': 'C.', 'label': 'PHI: NULL'}
{'fid': 'file5396', 'idx': 3183, 'content': 'There is no evidence of malignancy.', 'label': 'PHI: NULL'}
{'fid': '363', 'idx': 83, 'content': 'Arbor Park', 'label': 'STREET: Arbor Park'}
{'fid': 'file8473', 'idx': 4444, 'content': 'The ovary is sectioned transversely into 9 slices and shows a relatively well circumscribed 19 x 11 x approximately 20mm pale yellow lesion underlying the puckered serosal region (slices 4 to 8).', 'label': 'PHI: NULL'}


In [ ]:
PAD_IDX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
IGNORED_PAD_IDX = -100
PAD_IDX

50278

In [ ]:
from torch.utils.data import DataLoader
import torch

train_data = list(sub_datasets[0])

def collate_batch(batch):
    texts = [f"{bos} {data['content']} {sep}"+ data['label'].replace('\\n','\n')+f" {eos}" for data in list(batch)] # 範例 prompt
    encoded_seq = tokenizer(texts, padding=True)

    indexed_tks = torch.tensor(encoded_seq['input_ids'])
    attention_mask = torch.tensor(encoded_seq['attention_mask'])
    encoded_label = torch.tensor(encoded_seq['input_ids'])
    encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

    return indexed_tks, encoded_label, attention_mask

train_dataloader = DataLoader(train_data, batch_size=2, shuffle=False, collate_fn=collate_batch)
titer = iter(train_dataloader)
tks, labels, masks= next(titer)
print(tks.shape)
next(iter(titer))

torch.Size([2, 16])


(tensor([[50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278,
          50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278,
          50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278,
          50278, 50278, 50278, 50278, 50278, 50278, 50278,     0,  1780,  3399,
           4913,   209, 50279,  1267,  1848,  2025,    27,  1780,  3399,  4913,
            209, 50277],
         [    0,   380, 43735,   310,  2593,   264,   811, 13110,   715,   898,
          18484,   285,  2722,   247,  4942,   973,  4493, 31509,   655,  1269,
           1903,  1269,  5512,  1384,  2188, 14530,  8862, 15411,  6944,   253,
            268,  1028, 40617,  1151, 22399,  2919,   313,    84,   663,   265,
            577,   281,   854,   481,   209, 50279,  6663,    42,    27,  5812,
            209, 50277]]),
 tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -

In [ ]:
import random
BATCH_SIZE = 5 # 自行決定大小

class BatchSampler():
    def __init__(self, data, batch_size):
        self.pooled_indices = []
        self.data = data
        self.batch_size = batch_size
        self.len = len(list(data))
    def __iter__(self):
        self.pooled_indices = []
        indices = [(index, len(data["content"])) for index, data in enumerate(self.data)]
        random.shuffle(indices)
        for i in range(0, len(indices), BATCH_SIZE * 100):
            self.pooled_indices.extend(sorted(indices[i:i + BATCH_SIZE * 100], key=lambda x: x[1], reverse=True))
        self.pooled_indices = [x[0] for x in self.pooled_indices]

        for i in range(0, len(self.pooled_indices), BATCH_SIZE):
            yield self.pooled_indices[i:i + BATCH_SIZE]
    def __len__(self):
        return (self.len + self.batch_size - 1) // self.batch_size

bucket_train_dataloader = DataLoader(train_data, batch_sampler=BatchSampler(train_data, BATCH_SIZE),
                                     collate_fn=collate_batch, pin_memory=True)

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(plm,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
import torch
from tqdm import tqdm#, tqdm_notebook
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sample_text(model, tokenizer, seed, n_words=20):
    model = model.to(device)
    model.eval()
    text = tokenizer.encode(seed)
    inputs, past_key_values = torch.tensor([text]), None
    with torch.no_grad():
        for _ in tqdm(range(n_words)):
            out = model(inputs.to(device), past_key_values=past_key_values)
            logits = out.logits
            past_key_values = out.past_key_values
            log_probs = F.softmax(logits[:, -1], dim=-1)
            inputs = torch.multinomial(log_probs, 1)
            text.append(inputs.item())
            if tokenizer.decode(inputs.item()) == eos:
                break


    return tokenizer.decode(text)

sample_text(model, tokenizer, seed=f"{bos} DR AADLAND ABRAHAM {sep}")

100%|██████████| 20/20 [00:05<00:00,  3.35it/s]


'<|endoftext|> DR AADLAND ABRAHAM \n\n####\n\n Copyright folder Source "http://aws)\n\n## BSCB1839\n\nA member'

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

EPOCHS = 3 # 設定你的訓練次數
optimizer = AdamW(model.parameters(),lr=5e-5)

steps = len(bucket_train_dataloader)
total_steps = steps * EPOCHS
print(steps, total_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps*0.1,
    num_training_steps=total_steps
)

model.resize_token_embeddings(len(tokenizer))
model.to(device)
print(f'Total numbers of steps: {total_steps}')
model


17148 85740
Total numbers of steps: 85740


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
from os import write
from tqdm import tqdm,trange
import torch

global_step = 0
total_loss = 0

model.train()
for _ in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    predictions , true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels)#, attention_mask=masks)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

  # 指定保存路径和文件名
  save_path = '/content/drive/MyDrive/model.pth'

  # 保存模型的状态字典
  torch.save(model.state_dict(), save_path)

  print(f'Model saved at {save_path}')

In [ ]:
import torch

model_load_path = '/content/drive/MyDrive/160m_e3_all/model.pth'
model.load_state_dict(torch.load(model_load_path))
model.to(device)

print("Model has been loaded from {}".format(model_load_path))

Model has been loaded from /content/drive/MyDrive/160m_e3_all/model.pth


In [ ]:
from datasets import load_dataset, Features, Value
valid_data = load_dataset("csv", data_files="/content/drive/MyDrive/pre_data.txt", delimiter='\t',
                          features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'])
valid_list= list(valid_data['train'])
valid_list

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

[{'fid': '1001',
  'idx': 0,
  'content': 'Episode No:  88Y206206L',
  'label': None},
 {'fid': '1001', 'idx': 24, 'content': '8892062.BPL', 'label': None},
 {'fid': '1001',
  'idx': 37,
  'content': 'Vatterott, Jerrie CLARENCE',
  'label': None},
 {'fid': '1001',
  'idx': 65,
  'content': 'Lab No:  88Y20620,88Y20620',
  'label': None},
 {'fid': '1001', 'idx': 92, 'content': 'Exeter', 'label': None},
 {'fid': '1001',
  'idx': 99,
  'content': 'DECEPTION BAY  Northern Territory  6845',
  'label': None},
 {'fid': '1001',
  'idx': 139,
  'content': 'Specimen: Fluid,Tissue',
  'label': None},
 {'fid': '1001', 'idx': 162, 'content': 'D.O.B:  15/11/2004', 'label': None},
 {'fid': '1001', 'idx': 181, 'content': 'Sex:  F', 'label': None},
 {'fid': '1001',
  'idx': 189,
  'content': 'Collected: 20/5/2064 at :',
  'label': None},
 {'fid': '1001',
  'idx': 215,
  'content': 'Location:  PARKES 8 - GUNNEDAH DISTRICT HOSPITAL',
  'label': None},
 {'fid': '1001',
  'idx': 264,
  'content': 'DR Edison

In [ ]:
from tqdm import tqdm

PHI=frozenset({"PATIENT","DOCTOR","USERNAME","PROFESSION","ROOM","DEPARTMENT","HOSPITAL","ORGANIZATION","STREET","CITY","STATE","COUNTRY","ZIP",
        "LOCATION-OTHER","AGE","DATE","TIME","DURATION","SET","PHONE","FAX","EMAIL","URL",
        "IPADDR","SSN","MEDICALRECORD","HEALTHPLAN","ACCOUNT","LICENSE","VECHICLE","DEVICE","BIOID","IDNUM","OTHER"})

tokenizer.padding_side = 'left'
def sample_batch(model, tokenizer, input):
    """Generate text from a trained model."""

    model.eval()
    seeds = [f"{bos} {text['content']} {sep}" for text in input]
    texts = tokenizer(seeds, return_tensors = 'pt', padding=True).to(device)
    outputs = []
    #return
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**texts, max_new_tokens=400, pad_token_id = PAD_IDX,
                                        eos_token_id=tokenizer.convert_tokens_to_ids(eos))
        preds = tokenizer.batch_decode(output_tokens)
        temp1=0
        temp2=0
        temp3=0
        temp4=0
        temp5=0
        temp6=0
        count=0
        count2=0
        for idx , pred in enumerate(preds):
            pred = pred[pred.index(sep)+len(sep):].replace(pad, "").replace(eos, "").strip()
            if pred == "PHI: NULL":
                continue
            phis = pred.split('\n')
            lidxs = {}
            for p in phis:
                tid = p.find(':')
                if tid > 0:
                    text = p[tid+1:].strip()
                    nv = text.find('=>')
                    normalizedV = None # 處理時間正規化
                    if nv > 0:            # 如果箭頭符號存在，表示需要進行正規化
                      value = text[nv+2:].strip()  # 獲取箭頭後的部分，去除空白
                      normalizedV=value
                      text=text[:nv].strip()
                    lidx = 0
                    if text in lidxs:
                        lidx = lidxs[text]
                    lidx = input[idx]['content'].find(text, lidx)
                    eidx = lidx+len(text)
                    lidxs[text] = eidx
                    sidx=int(input[idx]['idx'])

                    if(count==0):                 #跟上上筆相同(偶)
                      temp3=int(lidx+sidx)
                      temp4=int(eidx+sidx)
                    if(count==2):
                      if(temp3==int(lidx+sidx) and temp4==int(eidx+sidx)):
                        temp3=int(lidx+sidx)
                        temp4=int(eidx+sidx)
                        count=1
                        continue
                      temp3=int(lidx+sidx)
                      temp4=int(eidx+sidx)
                      count=0

                    if(count2==1):                #跟上上筆相同(奇)
                      temp5=int(lidx+sidx)
                      temp6=int(eidx+sidx)
                    if(count2==3):
                      if(temp5==int(lidx+sidx) and temp6==int(eidx+sidx)):
                        temp5=int(lidx+sidx)
                        temp6=int(eidx+sidx)
                        count2=2
                        continue
                      temp5=int(lidx+sidx)
                      temp6=int(eidx+sidx)
                      count2=1
                    count+=1
                    count2+=1

                    if(temp1==int(lidx+sidx) and temp2==int(eidx+sidx)): #連續相同
                      continue
                    temp1=int(lidx+sidx)
                    temp2=int(eidx+sidx)

                    if(int(lidx+sidx)==int(eidx+sidx)): # 如果起點跟終點位置相同時
                      print("初始"+str(lidx+sidx)+"終點"+str(eidx+sidx))
                      continue

                    if(p[:tid] not in PHI):
                      print("NOT IN PHI"+p[:tid])
                      continue
                    if normalizedV is None:
                        outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}')
                    else:
                        outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}\t{normalizedV}')
    return outputs

f = open("answer.txt", "w", encoding="utf-8")
BATCH_SIZE = 8
for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
    with torch.no_grad():
        seeds = valid_list[i:i+BATCH_SIZE]
        outputs = sample_batch(model, tokenizer, input=seeds)
        for o in outputs:
            f.write(o)
            f.write('\n')
f.close()